In [ ]:
# prompt: necesito cargar datos de un csv que se llama export.csv de drive

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Thesis/export.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Contar los registros 'Is Laundering' en 0 y 1
counts = df['Is Laundering'].value_counts()

# Calcular los porcentajes
percentages = counts / counts.sum() * 100

# Imprimir los resultados
print(percentages)


Is Laundering
0    92.987261
1     7.012739
Name: count, dtype: float64


In [ ]:
import pandas as pd

# Supongamos que df_combined es tu DataFrame original
# Selecciona casos positivos
df_positivos = df[df['Is Laundering'] == 1]

# Selecciona casos negativos
df_negativos = df[df['Is Laundering'] == 0]

# Determina el número deseado de casos positivos y negativos
n_positivos = len(df_positivos)
n_negativos = int(1.5 * n_positivos)  # Ajusta esto para alcanzar el 70% negativos

# Reducción de casos negativos
df_negativos_reducidos = df_negativos.sample(n=n_negativos, random_state=42)

# Combina los datos balanceados
df_balanceado = pd.concat([df_positivos, df_negativos_reducidos])

# Baraja el DataFrame final
df_balanceado = df_balanceado.sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
# Contar los registros 'Is Laundering' en 0 y 1
counts = df_balanceado['Is Laundering'].value_counts()

# Calcular los porcentajes


# Imprimir los resultados
print(counts)

Is Laundering
0    573897
1    382598
Name: count, dtype: int64


In [ ]:
df_balanceado.dtypes

,0
Timestamp,object
From Bank,int64
Account2,object
To Bank,int64
Account4,object
Amount Received,float64
Receiving Currency,object
Amount Paid,float64
Payment Currency,object
Payment Format,object


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Ejemplo de codificación para una sola columna categórica
label_encoder = LabelEncoder()

# Codificar 'Account2'
df_balanceado['Account2'] = label_encoder.fit_transform(df_balanceado['Account2'])

# Si tienes múltiples columnas categóricas
for col in ['Account2', 'Receiving Currency', 'Payment Currency', 'Payment Format']:
    df_balanceado[col] = label_encoder.fit_transform(df_balanceado[col].astype(str))

In [ ]:
df_balanceado['Timestamp'] = pd.to_datetime(df_balanceado['Timestamp'], errors='coerce')  # Convertir a datetime

# Extraer características útiles
df_balanceado['Year'] = df_balanceado['Timestamp'].dt.year
df_balanceado['Month'] = df_balanceado['Timestamp'].dt.month
df_balanceado['Day'] = df_balanceado['Timestamp'].dt.day
df_balanceado['Hour'] = df_balanceado['Timestamp'].dt.hour
df_balanceado['Minute'] = df_balanceado['Timestamp'].dt.minute

# Eliminar la columna original de fecha si ya no es necesaria
df_balanceado = df_balanceado.drop(columns=['Timestamp'])

In [ ]:
# Verificar los primeros valores en la columna 'Account4'
print(df_balanceado['Account4'].head())

# Verificar los valores únicos para identificar posibles problemas
print(df_balanceado['Account4'].unique())


0    81492E1E1
1    8026170E0
2    8089CD9A0
3    8000EB430
4    8127BBDD0
Name: Account4, dtype: object
['81492E1E1' '8026170E0' '8089CD9A0' ... '80DCF06B0' '80220A090'
 '81B0DC560']


In [ ]:
label_encoder = LabelEncoder()
df_balanceado['Account4'] = label_encoder.fit_transform(df_balanceado['Account4'].astype(str))

In [ ]:
# Reemplazar valores no numéricos con NaN si es necesario
df_balanceado['Account4'] = pd.to_numeric(df_balanceado['Account4'], errors='coerce')

# Llenar valores nulos con la media de la columna
df_balanceado['Account4'].fillna(df_balanceado['Account4'].mean(), inplace=True)


In [ ]:
# Separar características y variable objetivo
X = df_balanceado.drop('Is Laundering', axis=1)
y = df_balanceado['Is Laundering']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)

# Hacer predicciones y evaluar el modelo
y_pred = rf_model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[113843    632]
 [  1800  75024]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99    114475
           1       0.99      0.98      0.98     76824

    accuracy                           0.99    191299
   macro avg       0.99      0.99      0.99    191299
weighted avg       0.99      0.99      0.99    191299



In [ ]:
# prompt: Convertir el modelo de Random forest  a un modelo entrenado archivo PKL y guardarlo en mi unidad de drive

import pickle

# Guardar el modelo entrenado en un archivo PKL
filename = '/content/drive/My Drive/Thesis/random_forest_model.pkl'
pickle.dump(rf_model, open(filename, 'wb'))
